# HW №1
## Task 1

In [1]:
%%capture
!pip3 install qiskit

In [2]:
import numpy as np
import qiskit as qk
from sympy import *

In [3]:
with open('./token', 'r') as token_file:
    token = token_file.read()

In [4]:
qk.IBMQ.save_account(token, overwrite = True)
qk.IBMQ.load_account()

/usr/local/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
provider = qk.IBMQ.get_provider(hub = 'ibm-q')

In [6]:
devices = provider.backends(filters=lambda x: (3 <= x.configuration().n_qubits <= 5) and not x.configuration().simulator)

In [7]:
real_backend = qk.providers.ibmq.least_busy(devices)
print(real_backend.configuration().n_qubits)

5


In [8]:
simd_backend = qk.Aer.get_backend('qasm_simulator')

## Task 2

Since we assume the following form of QBit wavefunction:
$\vert \psi \rangle = \sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle$

Actually we measure kinda projection on $Z$ - axis that corresponds $\theta$ angle. Thus $\phi$ angle describes projection onto $X,~Y$ axes. To fix it we can rotate system (or alternavely Vector) in the following way: $X,~Y,~Z \rightarrow Z,~X,~Y$. To do this we should choose axis of rotation, obviously it is normalized vector $(1,~1,~1)$ and angle to rotate $\alpha = 2 \pi / 3$.

To make rotation itself we should compute corresponding axis. We know the form of such rotations (it can be derived from infinitesimal rotations): $\hat{R}_{\vec{n}} (\alpha)= e^{-i \vec{n} \cdot \hat{\vec{\sigma}} \alpha / 2}$.
Since all sigma's satisfy $\hat{\sigma}_i^2 = \hat{I}$ it can be rewritten in the following form: 

$$\hat{R}_{\vec{n}} (\alpha) = I \cos{\frac{\alpha}{2}} - i \vec{n} \cdot \hat{\vec{\sigma}} \sin{\frac{\alpha}{2}}$$

Thus the function to compute such operator is:

In [9]:
px = np.array([[0, 1 ], [1 , 0]], dtype = 'complex128')
py = np.array([[0,-1j], [1j, 0]], dtype = 'complex128')
pz = np.array([[1, 0 ], [0 ,-1]], dtype = 'complex128')
pi = np.array([[1, 0 ], [0 , 1]], dtype = 'complex128')

In [10]:
def rot_mat(vec, angle):
    assert len(vec) == 3
    lvec = np.array(vec, dtype = 'float64')
    lvec = lvec / np.linalg.norm(lvec)
    mat = lvec[0] * px + lvec[1] * py + lvec[2] * pz
    sq_mat = mat @ mat
    assert np.max(np.abs(sq_mat - pi)) < 1e-4
    res = pi * np.cos(angle / 2) - 1j * mat * np.sin(angle / 2)
    sq_res = np.conjugate(res).T @ res
    assert np.max(np.abs(sq_res - pi)) < 1e-4
    return res

We needs not general but specific rotation:

In [11]:
rot = rot_mat([1, 1, 1], 2 * np.pi / 3)

In [12]:
print(rot)

[[ 0.5-0.5j -0.5-0.5j]
 [ 0.5-0.5j  0.5+0.5j]]


We need to choose some random (but normalized) initial state:

In [13]:
state = np.random.rand(2) + 1j * np.random.rand(2)
state = state / np.linalg.norm(state)
psi_angle = np.angle(state[0])
state = np.exp(-1j * psi_angle) * state
assert np.imag(state[0]) < 1.e-4

Since here state has form: 
$\vert \psi \rangle = e^{i\psi} \sin \theta \vert 0 \rangle + e^{i\phi} \cos \theta \vert 1 \rangle$
We can rewrite in the following form:
$\vert \psi \rangle = e^{i\psi} (\sin \theta \vert 0 \rangle + \exp{i\varphi} \cos \theta \vert 1 \rangle)$, thus $\varphi = \phi - \psi$.

In [38]:
varphi = np.angle(state[1])
theta  = np.arcsin(state[0])

In [39]:
varphi, theta

(0.8499274381374198, (0.3058488498816444+0j))

Let's compute symbolic result:

In [32]:
tht, phi = symbols('theta, phi', real = True)

In [33]:
vec0 = Matrix([sin(theta), exp(I * phi) * cos(tht)])

In [34]:
rot_op = Matrix([[1-I, -1-I], [1-I, 1+I]]) / 2

In [35]:
vec1 = rot_op @ vec0

In [36]:
ratio = ((conjugate(vec1[0]) * vec1[0]) / (conjugate(vec1[1]) * vec1[1])).simplify()

In [37]:
ratio

(I*exp(2*I*phi)*sin(2*theta) - 2*exp(I*phi) - I*sin(2*theta))/(-I*exp(2*I*phi)*sin(2*theta) - 2*exp(I*phi) + I*sin(2*theta))

As we can see ratio of probabilities is:
$$\frac{p(\vert 0 \rangle)}{p(\vert 1 \rangle)} = - \frac{e^{i\phi} (\sin{2\theta} \sin{\phi} + 1)}{e^{i\phi} (\sin{2\theta} \sin{\phi} - 1)} = r$$
Thus the output is:
$$\sin{\phi} \sin{2 \theta} = \frac{r - 1}{r + 1}$$

In [28]:
q = qk.QuantumRegister(1)
c = qk.ClassicalRegister(1)
qc = qk.QuantumCircuit(q, c)
qc.initialize(state, q)
qc.unitary(rot, q)
qc.measure(q, c)
res = qk.execute(qc, backend = simd_backend, shots = 65536).result()
r_simd = res.get_counts()

In [30]:
print(r_simd)

{'0': 46917, '1': 18619}


In [31]:
qc.draw()

┌─────────────────────────────────────┐┌─────────┐┌─┐
q0_0: ┤ initialize(0.3011,0.62941+0.71637j) ├┤ unitary ├┤M├
      └─────────────────────────────────────┘└─────────┘└╥┘
c0: 1/═══════════════════════════════════════════════════╩═
                                                         0

Let's compute $\sin \phi$:

In [40]:
r = r_simd['0'] / r_simd['1']
sin_phi = (r - 1) / (r + 1) / np.sin(2 * theta)

In [41]:
res_phi = np.arcsin(sin_phi)

In [42]:
res_phi

(0.8509617839983838+0j)